In [ ]:
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
def netCDF4_to_csv(nc4_key, nc4_url, yearN=1980): #Condition- function starts from year 1980, nc4_file_path -> str ,nc4_key -> str, datatype either monthly or yearly
    try:
        import requests
        import h5py
        import netCDF4
        from netCDF4 import Dataset
        import pandas as pd
        import numpy as np
        import datetime as dateformat
    except:
        print("Error - Required Libraries not installed")
        return
    
    try:
        r = requests.get(nc4_url, allow_redirects=True)
        open(nc4_key+'.nc4', 'wb').write(r.content)
        ds = Dataset(nc4_key+'.nc4')
    except:
        print('Error - Dataset not able to find the file')
        return
    
    
    if type(nc4_key) is not str:
        print('Error - Check input for key type')
        return
        
    if type(yearN) is not int:
        print('Error - Check input for year')
        return
    elif yearN < 0:
        print("Error- enter a positive number for year")
        return
        
    
    ### This will determine the number of years/months in the NC4 data   
    f = h5py.File(nc4_key+'.nc4', 'r')
    print(f.keys())
    size = f[nc4_key].shape
    num = size[0]
    
    key = ds[nc4_key][:]

    lat = ds['lat'][:]
    lon = ds['lon'][:]
    
    year = yearN
    count = 0

    # monthly data
    if num == 1392:
        start = (year - 1900 - 1)*12;
        datatype = 'Monthly_Output'
        print(datatype)
        
        #Jan of YearN to Dec of 2016
        for i in range(start,num):
            data = key[i,:,:]
            if i == start:
                month = 1
                data_pd = pd.DataFrame(data = data)
                lat = ds['lat'][:]
                data_pd.index =  lat
                lon = ds['lon'][:]
                data_pd.columns = lon
                data_pd = pd.melt(data_pd, ignore_index = False, var_name='lon', value_name=str(dateformat.datetime(year, 1, month).strftime('%m/%d/%Y')))               
            else:
                data_I = pd.DataFrame(data = data)
                data_I.index =  lat
                
                if i % 12 == 0:
                    year = year+1
                    month = 1
                else:
                    month = month + 1
                    
                data_I = pd.melt(data_I,ignore_index = False, value_name= str(dateformat.datetime(year, month,1).strftime('%m/%d/%Y')))
                data_pd[str(dateformat.datetime(year,month,1).strftime('%m/%d/%Y'))] = data_I.loc[:,str(dateformat.datetime(year,month,1).strftime('%m/%d/%Y'))]
        
        # Convert to Yearly Data
        data_ion = data_pd['lon']
        data_no_ion = data_pd.drop(columns=['lon'])
        new = (data_no_ion.groupby(pd.PeriodIndex(data_no_ion.columns, freq='Y'), axis=1).mean().rename(columns=lambda c: str(c).lower()))
        data_pd=pd.concat([data_ion, new], axis=1) 
        
    # yearly data
    elif num == 116:
        start = year - 1900 - 1;
        datatype = 'Yearly_Output'
        print(datatype)
        
        #YearN to 2016
        for i in range(start,num):
            data = key[i,:,:]
            if i == start:
                data_pd = pd.DataFrame(data = data)
                lat = ds['lat'][:]
                data_pd.index =  lat
                lon = ds['lon'][:]
                data_pd.columns = lon
                data_pd = pd.melt(data_pd, ignore_index = False, var_name='lon', value_name=str(yearN))
            else:
                data_I = pd.DataFrame(data = data)
                data_I.index =  lat
                year = year+1
                data_I = pd.melt(data_I,ignore_index = False, value_name= str(year))
                data_pd[str(year)] = data_I.loc[:,str(year)]
            
    data_pd.to_csv("watergap_"+nc4_key+"_"+datatype+".csv")
    return data_pd

In [ ]:
netCDF4_to_csv(nc4_key='anag', nc4_url='https://hs.pangaea.de/model/WaterGAP_v2-2d/watergap_22d_WFDEI-GPCC_histsoc_anag_monthly_1901_2016.nc4')
netCDF4_to_csv(nc4_key='anas', nc4_url='https://hs.pangaea.de/model/WaterGAP_v2-2d/watergap_22d_WFDEI-GPCC_histsoc_anas_monthly_1901_2016.nc4')
netCDF4_to_csv(nc4_key='atotuse', nc4_url='https://hs.pangaea.de/model/WaterGAP_v2-2d/watergap_22d_WFDEI-GPCC_histsoc_atotuse_monthly_1901_2016.nc4')
netCDF4_to_csv(nc4_key='canopystor', nc4_url='https://hs.pangaea.de/model/WaterGAP_v2-2d/watergap_22d_WFDEI-GPCC_histsoc_canopystor_monthly_1901_2016.nc4')
netCDF4_to_csv(nc4_key='dis', nc4_url='https://hs.pangaea.de/model/WaterGAP_v2-2d/watergap_22d_WFDEI-GPCC_histsoc_dis_monthly_1901_2016.nc4')
netCDF4_to_csv(nc4_key='evap', nc4_url='https://hs.pangaea.de/model/WaterGAP_v2-2d/watergap_22d_WFDEI-GPCC_histsoc_evap_monthly_1901_2016.nc4')
netCDF4_to_csv(nc4_key='glolakestor', nc4_url='https://hs.pangaea.de/model/WaterGAP_v2-2d/watergap_22d_WFDEI-GPCC_histsoc_glolakestor_monthly_1901_2016.nc4')
netCDF4_to_csv(nc4_key='glowetlandstor', nc4_url='https://hs.pangaea.de/model/WaterGAP_v2-2d/watergap_22d_WFDEI-GPCC_histsoc_glowetlandstor_monthly_1901_2016.nc4')
netCDF4_to_csv(nc4_key='groundwstor', nc4_url='https://hs.pangaea.de/model/WaterGAP_v2-2d/watergap_22d_WFDEI-GPCC_histsoc_groundwstor_monthly_1901_2016.nc4')
netCDF4_to_csv(nc4_key='loclakestor', nc4_url='https://hs.pangaea.de/model/WaterGAP_v2-2d/watergap_22d_WFDEI-GPCC_histsoc_loclakestor_monthly_1901_2016.nc4')
netCDF4_to_csv(nc4_key='locwetlandstor', nc4_url='https://hs.pangaea.de/model/WaterGAP_v2-2d/watergap_22d_WFDEI-GPCC_histsoc_locwetlandstor_monthly_1901_2016.nc4')
netCDF4_to_csv(nc4_key='ncrun', nc4_url='https://hs.pangaea.de/model/WaterGAP_v2-2d/watergap_22d_WFDEI-GPCC_histsoc_ncrun_monthly_1901_2016.nc4')
netCDF4_to_csv(nc4_key='pdomuse', nc4_url='https://hs.pangaea.de/model/WaterGAP_v2-2d/watergap_22d_WFDEI-GPCC_histsoc_pdomuse_yearly_1901_2016.nc4')
netCDF4_to_csv(nc4_key='pdomww', nc4_url='https://hs.pangaea.de/model/WaterGAP_v2-2d/watergap_22d_WFDEI-GPCC_histsoc_pdomww_yearly_1901_2016.nc4')
netCDF4_to_csv(nc4_key='pgwuse', nc4_url='https://hs.pangaea.de/model/WaterGAP_v2-2d/watergap_22d_WFDEI-GPCC_histsoc_pgwuse_monthly_1901_2016.nc4')
netCDF4_to_csv(nc4_key='pgwww', nc4_url='https://hs.pangaea.de/model/WaterGAP_v2-2d/watergap_22d_WFDEI-GPCC_histsoc_pgwww_monthly_1901_2016.nc4')
netCDF4_to_csv(nc4_key='pinduse', nc4_url='https://hs.pangaea.de/model/WaterGAP_v2-2d/watergap_22d_WFDEI-GPCC_histsoc_pinduse_yearly_1901_2016.nc4')
netCDF4_to_csv(nc4_key='pindww', nc4_url='https://hs.pangaea.de/model/WaterGAP_v2-2d/watergap_22d_WFDEI-GPCC_histsoc_pindww_yearly_1901_2016.nc4')
netCDF4_to_csv(nc4_key='pirruse', nc4_url='https://hs.pangaea.de/model/WaterGAP_v2-2d/watergap_22d_WFDEI-GPCC_histsoc_pirruse_monthly_1901_2016.nc4')
netCDF4_to_csv(nc4_key='pirrww', nc4_url='https://hs.pangaea.de/model/WaterGAP_v2-2d/watergap_22d_WFDEI-GPCC_histsoc_pirrww_monthly_1901_2016.nc4')
netCDF4_to_csv(nc4_key='plivuse', nc4_url='https://hs.pangaea.de/model/WaterGAP_v2-2d/watergap_22d_WFDEI-GPCC_histsoc_plivuse_yearly_1901_2016.nc4')
netCDF4_to_csv(nc4_key='pnag', nc4_url='https://hs.pangaea.de/model/WaterGAP_v2-2d/watergap_22d_WFDEI-GPCC_histsoc_pnag_monthly_1901_2016.nc4')
netCDF4_to_csv(nc4_key='pnas', nc4_url='https://hs.pangaea.de/model/WaterGAP_v2-2d/watergap_22d_WFDEI-GPCC_histsoc_pnas_monthly_1901_2016.nc4')
netCDF4_to_csv(nc4_key='potevap', nc4_url='https://hs.pangaea.de/model/WaterGAP_v2-2d/watergap_22d_WFDEI-GPCC_histsoc_potevap_monthly_1901_2016.nc4')
netCDF4_to_csv(nc4_key='precmon', nc4_url='https://hs.pangaea.de/model/WaterGAP_v2-2d/watergap_22d_WFDEI-GPCC_histsoc_precmon_monthly_1901_2016.nc4')
netCDF4_to_csv(nc4_key='ptotuse', nc4_url='https://hs.pangaea.de/model/WaterGAP_v2-2d/watergap_22d_WFDEI-GPCC_histsoc_ptotuse_monthly_1901_2016.nc4')
netCDF4_to_csv(nc4_key='ptotww', nc4_url='https://hs.pangaea.de/model/WaterGAP_v2-2d/watergap_22d_WFDEI-GPCC_histsoc_ptotww_monthly_1901_2016.nc4')
netCDF4_to_csv(nc4_key='qg', nc4_url='https://hs.pangaea.de/model/WaterGAP_v2-2d/watergap_22d_WFDEI-GPCC_histsoc_qg_monthly_1901_2016.nc4')
netCDF4_to_csv(nc4_key='ql', nc4_url='https://hs.pangaea.de/model/WaterGAP_v2-2d/watergap_22d_WFDEI-GPCC_histsoc_ql_monthly_1901_2016.nc4')
netCDF4_to_csv(nc4_key='qr', nc4_url='https://hs.pangaea.de/model/WaterGAP_v2-2d/watergap_22d_WFDEI-GPCC_histsoc_qr_monthly_1901_2016.nc4')
netCDF4_to_csv(nc4_key='qrdif', nc4_url='https://hs.pangaea.de/model/WaterGAP_v2-2d/watergap_22d_WFDEI-GPCC_histsoc_qrdif_monthly_1901_2016.nc4')
netCDF4_to_csv(nc4_key='qrswb', nc4_url='https://hs.pangaea.de/model/WaterGAP_v2-2d/watergap_22d_WFDEI-GPCC_histsoc_qrswb_monthly_1901_2016.nc4')
netCDF4_to_csv(nc4_key='qs', nc4_url='https://hs.pangaea.de/model/WaterGAP_v2-2d/watergap_22d_WFDEI-GPCC_histsoc_qs_monthly_1901_2016.nc4')
netCDF4_to_csv(nc4_key='reservoirstor', nc4_url='https://hs.pangaea.de/model/WaterGAP_v2-2d/watergap_22d_WFDEI-GPCC_histsoc_reservoirstor_monthly_1901_2016.nc4')
netCDF4_to_csv(nc4_key='riverstor', nc4_url='https://hs.pangaea.de/model/WaterGAP_v2-2d/watergap_22d_WFDEI-GPCC_histsoc_riverstor_monthly_1901_2016.nc4')
netCDF4_to_csv(nc4_key='soilmoist', nc4_url='https://hs.pangaea.de/model/WaterGAP_v2-2d/watergap_22d_WFDEI-GPCC_histsoc_soilmoist_monthly_1901_2016.nc4')
netCDF4_to_csv(nc4_key='swe', nc4_url='https://hs.pangaea.de/model/WaterGAP_v2-2d/watergap_22d_WFDEI-GPCC_histsoc_swe_monthly_1901_2016.nc4')
netCDF4_to_csv(nc4_key='tws', nc4_url='https://hs.pangaea.de/model/WaterGAP_v2-2d/watergap_22d_WFDEI-GPCC_histsoc_tws_monthly_1901_2016.nc4')
netCDF4_to_csv(nc4_key='WFDEI-GPCC_nosoc_disnat', nc4_url='https://hs.pangaea.de/model/WaterGAP_v2-2d/watergap_22d_WFDEI-GPCC_nosoc_disnat_monthly_1901_2016.nc4')
netCDF4_to_csv(nc4_key='WFDEI-GPCC_nosoc_ncrunnat', nc4_url='https://hs.pangaea.de/model/WaterGAP_v2-2d/watergap_22d_WFDEI-GPCC_nosoc_ncrunnat_monthly_1901_2016.nc4')

In [ ]:
#Converting csv to dataframe
df_anag = pd.read_csv(r'watergap_anag.csv')
df_anas = pd.read_csv(r'watergap_anas.csv')
df_atotuse = pd.read_csv(r'watergap_atotuse.csv')
df_canopystor = pd.read_csv(r'watergap_canopystor.csv')
df_dis = pd.read_csv(r'watergap_dis.csv')
df_evap = pd.read_csv(r'watergap_evap.csv')
df_glolakestor = pd.read_csv(r'watergap_glolakestor.csv')
df_glowetlandstor = pd.read_csv(r'watergap_glowetlandstor.csv')
df_groundwstor = pd.read_csv(r'watergap_groundwstor.csv')
df_loclakestor = pd.read_csv(r'watergap_loclakestor.csv')
df_locwetlandstor = pd.read_csv(r'watergap_locwetlandstor.csv')
df_ncrun = pd.read_csv(r'watergap_ncrun.csv')
df_pdomuse = pd.read_csv(r'watergap_pdomuse.csv')
df_pdomww = pd.read_csv(r'watergap_pdomww.csv')
df_pgwuse = pd.read_csv(r'watergap_pgwuse.csv')
df_pgwww = pd.read_csv(r'watergap_pgwww.csv')
df_pinduse = pd.read_csv(r'watergap_pinduse.csv')
df_pindww = pd.read_csv(r'watergap_pindww.csv')
df_pirruse = pd.read_csv(r'watergap_pirruse.csv')
df_pirrww = pd.read_csv(r'watergap_pirrww.csv')
df_plivuse = pd.read_csv(r'watergap_plivuse.csv')
df_pnag = pd.read_csv(r'watergap_pnag.csv')
df_pnas = pd.read_csv(r'watergap_pnas.csv')
df_potevap = pd.read_csv(r'watergap_potevap.csv')
df_precmon = pd.read_csv(r'watergap_precmon.csv')
df_ptotuse = pd.read_csv(r'watergap_ptotuse.csv')
df_ptotww = pd.read_csv(r'watergap_ptotww.csv')
df_qg = pd.read_csv(r'watergap_qg.csv')
df_ql = pd.read_csv(r'watergap_ql.csv')
df_qr = pd.read_csv(r'watergap_qr.csv')
df_qrdif = pd.read_csv(r'watergap_qrdif.csv')
df_qrswb = pd.read_csv(r'watergap_qrswb.csv')
df_qs = pd.read_csv(r'watergap_qs.csv')
df_reservoirstor = pd.read_csv(r'watergap_reservoirstor.csv')
df_riverstor = pd.read_csv(r'watergap_riverstor.csv')
df_soilmoist = pd.read_csv(r'watergap_soilmoist.csv')
df_swe = pd.read_csv(r'watergap_swe.csv')
df_tws = pd.read_csv(r'watergap_tws.csv')
df_WFDEI-GPCC_nosoc_disnat = pd.read_csv(r'watergap_WFDEI-GPCC_nosoc_disnat.csv')
df_WFDEI-GPCC_nosoc_ncrunnat = pd.read_csv(r'watergap_WFDEI-GPCC_nosoc_ncrunnat.csv')

In [ ]:
#Fixing the lat column
df_anag.rename(columns={'Unnamed: 0' : 'lat'}, inplace=True)
df_anas.rename(columns={'Unnamed: 0' : 'lat'}, inplace=True)
df_atotuse.rename(columns={'Unnamed: 0' : 'lat'}, inplace=True)
df_canopystor.rename(columns={'Unnamed: 0' : 'lat'}, inplace=True)
df_dis.rename(columns={'Unnamed: 0' : 'lat'}, inplace=True)
df_evap.rename(columns={'Unnamed: 0' : 'lat'}, inplace=True)
df_glolakestor.rename(columns={'Unnamed: 0' : 'lat'}, inplace=True)
df_glowetlandstor.rename(columns={'Unnamed: 0' : 'lat'}, inplace=True)
df_groundwstor.rename(columns={'Unnamed: 0' : 'lat'}, inplace=True)
df_loclakestor.rename(columns={'Unnamed: 0' : 'lat'}, inplace=True)
df_locwetlandstor.rename(columns={'Unnamed: 0' : 'lat'}, inplace=True)
df_ncrun.rename(columns={'Unnamed: 0' : 'lat'}, inplace=True)
df_pdomuse.rename(columns={'Unnamed: 0' : 'lat'}, inplace=True)
df_pdomww.rename(columns={'Unnamed: 0' : 'lat'}, inplace=True)
df_pgwuse.rename(columns={'Unnamed: 0' : 'lat'}, inplace=True)
df_pgwww.rename(columns={'Unnamed: 0' : 'lat'}, inplace=True)
df_pinduse.rename(columns={'Unnamed: 0' : 'lat'}, inplace=True)
df_pindww.rename(columns={'Unnamed: 0' : 'lat'}, inplace=True)
df_pirruse.rename(columns={'Unnamed: 0' : 'lat'}, inplace=True)
df_pirrww.rename(columns={'Unnamed: 0' : 'lat'}, inplace=True)
df_plivuse.rename(columns={'Unnamed: 0' : 'lat'}, inplace=True)
df_pnag.rename(columns={'Unnamed: 0' : 'lat'}, inplace=True)
df_pnas.rename(columns={'Unnamed: 0' : 'lat'}, inplace=True)
df_potevap.rename(columns={'Unnamed: 0' : 'lat'}, inplace=True)
df_precmon.rename(columns={'Unnamed: 0' : 'lat'}, inplace=True)
df_ptotuse.rename(columns={'Unnamed: 0' : 'lat'}, inplace=True)
df_ptotww.rename(columns={'Unnamed: 0' : 'lat'}, inplace=True)
df_qg.rename(columns={'Unnamed: 0' : 'lat'}, inplace=True)
df_ql.rename(columns={'Unnamed: 0' : 'lat'}, inplace=True)
df_qr.rename(columns={'Unnamed: 0' : 'lat'}, inplace=True)
df_qrdif.rename(columns={'Unnamed: 0' : 'lat'}, inplace=True)
df_qrswb.rename(columns={'Unnamed: 0' : 'lat'}, inplace=True)
df_qs.rename(columns={'Unnamed: 0' : 'lat'}, inplace=True)
df_reservoirstor.rename(columns={'Unnamed: 0' : 'lat'}, inplace=True)
df_riverstor.rename(columns={'Unnamed: 0' : 'lat'}, inplace=True)
df_soilmoist.rename(columns={'Unnamed: 0' : 'lat'}, inplace=True)
df_swe.rename(columns={'Unnamed: 0' : 'lat'}, inplace=True)
df_tws.rename(columns={'Unnamed: 0' : 'lat'}, inplace=True)
df_WFDEI-GPCC_nosoc_disnat.rename(columns={'Unnamed: 0' : 'lat'}, inplace=True)
df_WFDEI-GPCC_nosoc_ncrunnat.rename(columns={'Unnamed: 0' : 'lat'}, inplace=True)

In [ ]:
# add the variable's name to each of the years so we can identify each of them when we see the final dataframe.
df_anag = df_anag.add_suffix(_'anag')
df_anas = df_anas.add_suffix(_'anas')
df_atotuse = df_atotuse.add_suffix(_'atotuse')
df_canopystor = df_canopystor.add_suffix(_'canopystor')
df_dis = df_dis.add_suffix(_'dis')
df_evap = df_evap.add_suffix(_'evap')
df_glolakestor = df_glolakestor.add_suffix(_'glolakestor')
df_glowetlandstor = df_glowetlandstor.add_suffix(_'glowetlandstor')
df_groundwstor = df_groundwstor.add_suffix(_'groundwstor')
df_loclakestor = df_loclakestor.add_suffix(_'loclakestor')
df_locwetlandstor = df_locwetlandstor.add_suffix(_'locwetlandstor')
df_ncrun = df_ncrun.add_suffix(_'ncrun')
df_pdomuse = df_pdomuse.add_suffix(_'pdomuse')
df_pdomww = df_pdomww.add_suffix(_'pdomww')
df_pgwuse = df_pgwuse.add_suffix(_'pgwuse')
df_pgwww = df_pgwww.add_suffix(_'pgwww')
df_pinduse = df_pinduse.add_suffix(_'pinduse')
df_pindww = df_pindww.add_suffix(_'pindww')
df_pirruse = df_pirruse.add_suffix(_'pirruse')
df_pirrww = df_pirrww.add_suffix(_'pirrww')
df_plivuse = df_plivuse.add_suffix(_'plivuse')
df_pnag = df_pnag.add_suffix(_'pnag')
df_pnas = df_pnas.add_suffix(_'pnas')
df_potevap = df_potevap.add_suffix(_'potevap')
df_precmon = df_precmon.add_suffix(_'precmon')
df_ptotuse = df_ptotuse.add_suffix(_'ptotuse')
df_ptotww = df_ptotww.add_suffix(_'ptotww')
df_qg = df_qg.add_suffix(_'qg')
df_ql = df_ql.add_suffix(_'ql')
df_qr = df_qr.add_suffix(_'qr')
df_qrdif = df_qrdif.add_suffix(_'qrdif')
df_qrswb = df_qrswb.add_suffix(_'qrswb')
df_qs = df_qs.add_suffix(_'qs')
df_reservoirstor = df_reservoirstor.add_suffix(_'reservoirstor')
df_riverstor = df_riverstor.add_suffix(_'riverstor')
df_soilmoist = df_soilmoist.add_suffix(_'soilmoist')
df_swe = df_swe.add_suffix(_'swe')
df_tws = df_tws.add_suffix(_'tws')
df_WFDEI-GPCC_nosoc_disnat = df_WFDEI-GPCC_nosoc_disnat.add_suffix(_'WFDEI-GPCC_nosoc_disnat')
df_WFDEI-GPCC_nosoc_ncrunnat = df_WFDEI-GPCC_nosoc_ncrunnat.add_suffix(_'WFDEI-GPCC_nosoc_ncrunnat')

In [ ]:
# Concatenate all dataframes
df_yearly_concat = pd.concat([df_anag, df_anas, df_atotuse, df_canopystor, df_dis, df_evap, df_glolakestor, df_glowetlandstor, df_groundwstor, df_loclakestor, df_locwetlandstor, df_ncrun, df_pdomuse, df_pdomww, df_pgwuse, df_pgwww, df_pinduse, df_pindww, df_pirruse, df_pirrww, df_plivuse, df_pnag, df_pnas, df_potevap, df_precmon, df_ptotuse, df_ptotww, df_qg, df_ql, df_qr, df_qrdif, df_qrswb, df_qs, df_reservoirstor, df_riverstor, df_soilmoist, df_swe, df_tws, df_WFDEI-GPCC_nosoc_disnat, df_WFDEI-GPCC_nosoc_ncrunnat], axis=1)

In [ ]:
#drop the lat and lon columns in all other variables except anag var
to_drop = ['lat_anas', 'lat_atotuse', 'lat_canopystor', 'lat_dis', 'lat_evap', 'lat_glolakestor', 'lat_glowetlandstor', 'lat_groundwstor', 'lat_loclakestor', 'lat_locwetlandstor', 'lat_ncrun', 'lat_pdomuse', 'lat_pdomww', 'lat_pgwuse', 'lat_pgwww', 'lat_pinduse', 'lat_pindww', 'lat_pirruse', 'lat_pirrww', 'lat_plivuse', 'lat_pnag', 'lat_pnas', 'lat_potevap', 'lat_precmon', 'lat_ptotuse', 'lat_ptotww', 'lat_qg', 'lat_ql', 'lat_qr', 'lat_qrdif', 'lat_qrswb', 'lat_qs', 'lat_reservoirstor', 'lat_riverstor', 'lat_soilmoist', 'lat_swe', 'lat_tws', 'lat_WFDEI-GPCC_nosoc_disnat', 'lat_WFDEI-GPCC_nosoc_ncrunnat', 'lon_anas', 'lon_atotuse', 'lon_canopystor', 'lon_dis', 'lon_evap', 'lon_glolakestor', 'lon_glowetlandstor', 'lon_groundwstor', 'lon_loclakestor', 'lon_locwetlandstor', 'lon_ncrun', 'lon_pdomuse', 'lon_pdomww', 'lon_pgwuse', 'lon_pgwww', 'lon_pinduse', 'lon_pindww', 'lon_pirruse', 'lon_pirrww', 'lon_plivuse', 'lon_pnag', 'lon_pnas', 'lon_potevap', 'lon_precmon', 'lon_ptotuse', 'lon_ptotww', 'lon_qg', 'lon_ql', 'lon_qr', 'lon_qrdif', 'lon_qrswb', 'lon_qs', 'lon_reservoirstor', 'lon_riverstor', 'lon_soilmoist', 'lon_swe', 'lon_tws', 'lon_WFDEI-GPCC_nosoc_disnat', 'lon_WFDEI-GPCC_nosoc_ncrunnat']

df_yearly_concat.drop(to_drop, inplace=True, axis=1)

In [ ]:
# Final Touch: renaming lat and lon to general columns:
df_yearly_concat.rename(columns={'lat_anag' : 'lat', 'lon_anag' : 'lon' }, inplace=True)

In [ ]:
# COnverting to CSV
df_yearly_concat.to_csv("COMBINED_watergap.csv")